In [1]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/bach'

Mounted at /content/drive


In [2]:
!pip install wandb

In [3]:
import pandas as pd
import numpy as np

import os
import random
from glob import glob
from queue import Queue
from threading import Thread
from typing import Dict, List, Tuple, Optional

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.tv_tensors._image import Image

from dataclasses import dataclass

import wandb
from tqdm.auto import tqdm




In [4]:
PROCESSED_IMAGES_PATH = base_path + "/processed_images"
HIERARCHY_FILE_PATH = base_path + "/hierarchy.csv"

In [5]:
class Hierarchy():
    def __init__(self, path: Optional[str] = None):
        if path is None:
            path = HIERARCHY_FILE_PATH
        self.hierarchy: pd.DataFrame = pd.read_csv(path)

    def get_root_id(self):
        root_node = self.hierarchy[self.hierarchy["parent_id"].isnull()]

        return root_node["id"].values[0]

    def get_children(self, parent_id: str):

        children = self.hierarchy[self.hierarchy["parent_id"] == parent_id]

        return children["id"].tolist()

    def get_leaf_nodes(self, root_id: str) -> List[str]:
        """
        Recursively get all leaf nodes ids under a given root node
        """

        children = self.get_children(root_id)

        if not children:
            return [root_id]

        leaf_nodes = []

        for child in children:
            leaf_nodes.extend(self.get_leaf_nodes(child))

        return leaf_nodes


In [6]:
class ImageDataset(Dataset):
    def __init__(self, root_dir: str, categories: Dict[str, List[str]], split: str = 'train',
                 train_ratio: float = 0.70, val_ratio: float = 0.15):

        self.root_dir = os.path.normpath(root_dir)
        self.categories = categories
        self.split = split

        # Create category to index mapping
        self.cat_mapping = {cat: idx for idx,
                            cat in enumerate(categories.keys())}

        self.train_augmentations = v2.Compose([
            v2.RandomApply([
                v2.ColorJitter(
                    brightness=0.3,
                    contrast=0.3,
                    saturation=0.3,
                    hue=0.15
                )
            ], p=0.5),
            v2.RandomApply([
                v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
            ], p=0.4),
            v2.RandomApply([
                v2.RandomAdjustSharpness(sharpness_factor=2)
            ], p=0.4),
            v2.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)),
            v2.RandomApply([
                v2.RandomPerspective(distortion_scale=0.3)
            ], p=0.3),
            v2.RandomApply([
                v2.RandomRotation(degrees=15)
            ], p=0.3),
            v2.RandomHorizontalFlip(p=0.5),
            v2.RandomAutocontrast(p=0.3),
        ])

        self.rebalancing_augmentations = [
            'flip_h',         # Horizontal flip
            'flip_v',         # Vertical flip
            'rot90',          # 90 degree rotation
            'rot180',         # 180 degree rotation
            'rot270',         # 270 degree rotation
            'flip_h_rot90',   # Combine flip and rotation
            'flip_v_rot90',   # Another combination
            'identity'        # No change
        ]

        # Collect all file paths and their categories
        self.samples = []

        n_samples = {}
        for cat, leaves in categories.items():
            n_samples[cat] = 0
            for leaf in leaves:
                cat_dir = os.path.join(self.root_dir, leaf)
                tensor_files = glob(os.path.join(cat_dir, '*.pt'))

                # Generate deterministic train/val/test split
                n_files = len(tensor_files)
                indices = list(range(n_files))

                # Seed random number generator for reproducibility
                random.Random(42).shuffle(indices)

                n_train = int(n_files * train_ratio)
                n_val = int(n_files * val_ratio)

                if split == 'train':
                    selected_indices = indices[:n_train]
                elif split == 'val':
                    selected_indices = indices[n_train:n_train + n_val]
                else:  # test
                    selected_indices = indices[n_train + n_val:]

                n_samples[cat] += len(selected_indices)
                for idx in selected_indices:
                    self.samples.append({
                        'path': tensor_files[idx],
                        'category': cat,
                        'label': self.cat_mapping[cat]
                    })

        # print sample distribution
        # percentage of samples in each category

        total_samples = sum(n_samples.values())

        for cat, n in n_samples.items():
            print(f"{cat}: {n} samples ({n/total_samples:.2%})")

        # Only balance classes for training data
        if split != 'train':
            return

        # Balance classes by creating transformations of underrepresented classes
        max_cat = max(n_samples, key=n_samples.get)
        max_cat_samples = max(n_samples.values())

        # create tmp path for storing transformed tensors
        tmp_dir = os.path.join(self.root_dir, 'tmp')
        os.makedirs(tmp_dir, exist_ok=True)

        # Balance classes by creating transformations
        for cat in self.categories.keys():
            if cat == max_cat:
                print(
                    f"Skipping balanced class {cat}, already has {max_cat_samples} samples")
                continue

            cat_samples = [
                sample for sample in self.samples if sample['category'] == cat]
            n_samples_cat = n_samples[cat]
            while n_samples_cat < max_cat_samples:
                # Select a random sample from the class
                sample = random.choice(cat_samples)
                # Create a random transformation
                tensor = torch.load(sample['path'], weights_only=True)

                transformation_type = random.choice(
                    self.rebalancing_augmentations)
                if transformation_type == 'flip_h':
                    transformed_tensor = torch.flip(
                        tensor, [2])  # Horizontal flip
                elif transformation_type == 'flip_v':
                    transformed_tensor = torch.flip(
                        tensor, [1])  # Vertical flip
                elif transformation_type == 'rot90':
                    transformed_tensor = torch.rot90(tensor, k=1, dims=(1, 2))
                elif transformation_type == 'rot180':
                    transformed_tensor = torch.rot90(tensor, k=2, dims=(1, 2))
                elif transformation_type == 'rot270':
                    transformed_tensor = torch.rot90(tensor, k=3, dims=(1, 2))
                elif transformation_type == 'flip_h_rot90':
                    transformed_tensor = torch.rot90(
                        torch.flip(tensor, [2]), k=1, dims=(1, 2))
                elif transformation_type == 'flip_v_rot90':
                    transformed_tensor = torch.rot90(
                        torch.flip(tensor, [1]), k=1, dims=(1, 2))
                else:  # identity
                    transformed_tensor = tensor

                # Save the transformed tensor to a temporary path
                tmp_path = os.path.join(
                    tmp_dir, f'{n_samples_cat}_{cat}.pt')

                assert transformed_tensor.shape == tensor.shape, \
                    f"Transformed tensor shape {transformed_tensor.shape} does not match original tensor shape {tensor.shape}"

                torch.save(transformed_tensor, tmp_path)

                self.samples.append({
                    'path': tmp_path,
                    'category': cat,
                    'label': self.cat_mapping[cat]
                })

                n_samples_cat += 1

            print(f"Balanced class {cat} to {n_samples_cat} samples")

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        sample = self.samples[idx]
        # Load preprocessed tensor
        tensor = torch.load(sample['path'], weights_only=True)

        if self.split == 'train':
            old_shape = tensor.shape
            tensor = self.train_augmentations(tensor)

            assert tensor.shape == old_shape, \
                f"Augmented tensor shape {tensor.shape} does not match original tensor shape {old_shape}"

        return tensor, torch.tensor(sample['label'], dtype=torch.long)


class PrefetchLoader:
    def __init__(self, loader: DataLoader, buffer_size: int = 2, device: torch.device = None, root_dir: str = None):
        self.loader = loader
        self.buffer_size = buffer_size
        self.device = device or torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')
        self.buffer = Queue(maxsize=buffer_size)
        self.stop_event = None
        self.prefetch_thread = None
        self.root_dir = root_dir
        self._active = False

    def prefetch_worker(self):
        try:
            for batch in self.loader:
                if self.stop_event.is_set():
                    break

                # Move batch to device in the background thread
                if isinstance(batch, (tuple, list)):
                    batch = tuple(t.to(self.device, non_blocking=True)
                                  if isinstance(t, torch.Tensor) else t
                                  for t in batch)
                elif isinstance(batch, dict):
                    batch = {k: v.to(self.device, non_blocking=True)
                             if isinstance(v, torch.Tensor) else v
                             for k, v in batch.items()}
                elif isinstance(batch, torch.Tensor):
                    batch = batch.to(self.device, non_blocking=True)

                self.buffer.put(batch)
        except Exception as e:
            print(f"Prefetch worker error: {e}")
            self.stop_event.set()
        finally:
            self.buffer.put(None)  # Signal end of data
            self._active = False

    def __iter__(self):
        # Clean up previous iteration if necessary
        if self._active:
            self.stop_event.set()
            if self.prefetch_thread is not None:
                self.prefetch_thread.join()
            while not self.buffer.empty():
                self.buffer.get()

        # Start new iteration
        self._active = True
        self.stop_event = torch.multiprocessing.Event()
        self.prefetch_thread = Thread(
            target=self.prefetch_worker,
            daemon=True
        )
        self.prefetch_thread.start()

        while True:
            if self.stop_event.is_set():
                raise RuntimeError("Prefetch worker encountered an error")

            batch = self.buffer.get()
            if batch is None:
                break
            yield batch

    def __len__(self):
        return len(self.loader)

    def __del__(self):
        if self._active:
            self.stop_event.set()
            if self.prefetch_thread is not None:
                self.prefetch_thread.join()

    def remove_tmp_folder(self):
        tmp_dir = os.path.join(self.root_dir, 'tmp')
        for file in os.listdir(tmp_dir):
            os.remove(os.path.join(tmp_dir, file))

        os.rmdir(tmp_dir)


def create_images_dataloader(
    categories: Dict[str, List[str]],
    batch_size: int = 32,
    split: str = 'train',
    num_workers: int = 4,
    n_prefetch_batches: int = 3,
    device: torch.device = None
) -> PrefetchLoader:
    """
    Creates a dataloader with prefetching for the specified categories.

    Args:
        root_dir: Path to processed data directory
        categories: List of category names to include
        batch_size: Batch size
        split: One of 'train', 'val', 'test'
        num_workers: Number of worker processes
        prefetch_factor: Number of batches to prefetch
    """

    dataset = ImageDataset(PROCESSED_IMAGES_PATH, categories, split)

    # Create base DataLoader
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(split == 'train'),
        num_workers=num_workers,
        drop_last=True,
    )

    # Wrap with prefetching
    return PrefetchLoader(loader, buffer_size=n_prefetch_batches, device=device, root_dir=PROCESSED_IMAGES_PATH)



In [7]:
import torch.nn as nn


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_rate=0.2):
        super().__init__()

        self.dropout = nn.Dropout2d(dropout_rate)

        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=3, stride=stride, padding=1)

        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, padding=1)

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride)
            )
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        # Pre-activation pattern
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)

        out += self.shortcut(x)

        return out


class HierarchyNodeModel(nn.Module):
    def __init__(self, num_classes, input_channels=3):
        super().__init__()


        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Wider but keep block count
        self.layer1 = nn.Sequential(
            ResidualBlock(16, 48, dropout_rate=0.1),
            ResidualBlock(48, 48, dropout_rate=0.1),
            ResidualBlock(48, 48, dropout_rate=0.1)
        )

        self.layer2 = nn.Sequential(
            ResidualBlock(48, 96, stride=2, dropout_rate=0.15),
            ResidualBlock(96, 96, dropout_rate=0.15),
            ResidualBlock(96, 96, dropout_rate=0.15)
        )

        self.layer3 = nn.Sequential(
            ResidualBlock(96, 192, stride=2, dropout_rate=0.2),
            ResidualBlock(192, 192, dropout_rate=0.2)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Slightly modified head
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(192, 96),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(96, num_classes)
        )

        # Weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(
                m.weight, mode='fan_in', nonlinearity='relu')
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = self.head(x)
        return x


In [8]:
@dataclass
class TrainConfig:
    # train parameters
    epochs: int
    batch_size: int

    # lr parameters
    max_lr: float
    div_factor: float
    final_div_factor: float
    pct_start: float

    # regularization parameters
    grad_clip_value: float
    weight_decay: float
    early_stopping_patience: int
    label_smoothing: float

    optimizer: str

    @property
    def initial_lr(self) -> float:
        return self.max_lr / self.div_factor

    @property
    def min_lr(self) -> float:
        return self.max_lr / self.final_div_factor


In [9]:
def __get_data_loaders(hierarchy: Hierarchy, node_id: str, train_config: TrainConfig) -> Tuple[DataLoader, DataLoader, int]:
    children = hierarchy.get_children(node_id)
    categories_dict = {child: hierarchy.get_leaf_nodes(
        child) for child in children}

    train_loader = create_images_dataloader(
        categories_dict, split='train', batch_size=train_config.batch_size)
    val_loader = create_images_dataloader(
        categories_dict, split='val', batch_size=train_config.batch_size)

    return train_loader, val_loader, len(children)


def __create_optimizer(model: nn.Module, train_config: TrainConfig) -> torch.optim.Optimizer:
    if train_config.optimizer == 'adamw':
        return torch.optim.AdamW(
            model.parameters(),
            lr=train_config.initial_lr,
            weight_decay=train_config.weight_decay
        )
    elif train_config.optimizer == 'adam':
        return torch.optim.Adam(
            model.parameters(),
            lr=train_config.initial_lr
        )
    else:
        raise ValueError(f'Invalid optimizer: {train_config.optimizer}')


class MetricSmoother:
    def __init__(self, window_size: int = 3):
        self.window_size = window_size
        self.values = []

    def update(self, value: float) -> float:
        self.values.append(value)
        if len(self.values) > self.window_size:
            self.values.pop(0)
        return self.get_smoothed()

    def get_smoothed(self) -> float:
        return sum(self.values) / len(self.values)


def get_mixup_alpha(train_acc, val_acc, epoch, total_epochs, grad_norm):
    """Dynamically adjust mixup alpha based on overfitting gap"""
    gap = train_acc - val_acc
# Adjust alpha based on gradient norm
    if grad_norm < 0.01:  # Very small gradients
        base_alpha = 0.4  # Less mixing to allow bigger steps
    elif grad_norm > 2.0:  # Large gradients
        base_alpha = 0.6  # More mixing for stability
    elif grad_norm > 1.5:
        base_alpha = 0.7
    elif grad_norm > 1.0:
        base_alpha = 0.8
    else:
        base_alpha = 0.6

    # Adjust for accuracy gap
    if gap > 0.15:
        base_alpha *= 1.2

    # Early training adjustment
    if epoch < total_epochs * 0.1:
        return min(1.0, base_alpha * 1.1)
    return base_alpha


def mixup_data(x, y, alpha):
    """Performs mixup on the input and target"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def get_gradient_norm(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5


In [10]:
def __train_epoch(
    model: nn.Module,
    loader: PrefetchLoader,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler.OneCycleLR,
    grad_clip_value: float = 0.5,
    label_smoothing: float = 0.1,
    prev_train_acc: float = 0.0,
    prev_val_acc: float = 0.0,
    epoch: int = 0,
    total_epochs: int = 100,
    avg_grad_norm: float = 0.0
) -> Tuple[float, float, float, float]:

    model.train()
    total_loss = 0
    correct = 0
    total_samples = 0

    alpha = get_mixup_alpha(prev_train_acc, prev_val_acc,
                            epoch, total_epochs, avg_grad_norm)
    print(f"Mixup alpha: {alpha}")

    progress_bar = tqdm(loader, desc='Training')
    grad_norms = []

    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    for batch_idx, (X_batch, y_batch) in enumerate(progress_bar):
        optimizer.zero_grad()

        mixed_X, y_a, y_b, lam = mixup_data(X_batch, y_batch, alpha)

        y_pred = model(mixed_X)

        loss = lam * criterion(y_pred, y_a) + (1 - lam) * \
            criterion(y_pred, y_b)

        loss.backward()

        grad_norm = get_gradient_norm(model)
        grad_norms.append(grad_norm)

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), grad_clip_value, norm_type=2)

        optimizer.step()
        scheduler.step()

        # Update metrics
        model.eval()
        with torch.no_grad():
            original_outputs = model(X_batch)
            pred = original_outputs.argmax(dim=1)
            correct += (pred == y_batch).sum().item()
        model.train()

        total_samples += y_batch.size(0)
        total_loss += loss.item()

        # Update progress bar
        progress_bar.set_postfix(
            loss=f"{loss.item():.4f}",
            acc=f"{100.0 * correct / total_samples:.2f}%"
        )

    epoch_loss = total_loss / len(loader)
    epoch_acc = correct / total_samples

    avg_grad_norm = np.mean(grad_norms)

    return epoch_loss, epoch_acc, alpha, avg_grad_norm





In [11]:
@torch.no_grad()
def __validate(model: nn.Module, loader: PrefetchLoader) -> Tuple[float, float]:
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0

    criterion = nn.CrossEntropyLoss()
    progress_bar = tqdm(
        loader, desc='Validation')
    for X_batch, y_batch in progress_bar:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)

        total_loss += loss.item()
        correct += (y_pred.argmax(dim=1) == y_batch).sum().item()
        total_samples += y_batch.size(0)

        progress_bar.set_postfix(
            loss=f"{loss.item():.4f}",
            acc=f"{100.0 * correct / total_samples:.2f}%"
        )

    epoch_loss = total_loss / len(loader)
    epoch_acc = correct / total_samples
    return epoch_loss, epoch_acc



In [12]:
from datetime import datetime
from torchvision.transforms import v2


In [13]:
def train_singular_model(hierarchy: Hierarchy, node_id: str, train_config: TrainConfig, device: str) -> None:

    # init wandb run
    run_name = f"node_{node_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    wandb.init(
        project="bachelor-resnet-icaam",
        name=run_name,
        config={
            "architecture": "HierarchyNodeModel",
            "epochs": train_config.epochs,
            "batch_size": train_config.batch_size,
            "optimizer": train_config.optimizer,
            "weight_decay": train_config.weight_decay,
            "grad_clip_value": train_config.grad_clip_value,
            "label_smoothing": train_config.label_smoothing,
            "early_stopping_patience": train_config.early_stopping_patience,
            "pct_start": train_config.pct_start,
            "div_factor": train_config.div_factor,
            "final_div_factor": train_config.final_div_factor,
            "max_lr": train_config.max_lr,
            "node_id": node_id,
            "num_classes": None  # Will be set after loading data
        }
    )
    # Setup data
    train_loader, val_loader, num_classes = __get_data_loaders(
        hierarchy, node_id, train_config)

    # Update wandb config
    wandb.config.update({"num_classes": num_classes}, allow_val_change=True)

    # Setup logging
    #logger = create_file_logger(f'train_{node_id}.log')
    #logger.info(f'Training model for node {node_id}')

    # Create model
    model = HierarchyNodeModel(num_classes=num_classes)
    model.to(device)

    wandb.watch(model, log="all", log_freq=10)

    # Setup training
    optimizer = __create_optimizer(model, train_config)

    # Setup schedulers
    num_steps = len(train_loader) * train_config.epochs
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=train_config.max_lr,
        total_steps=num_steps,
        pct_start=train_config.pct_start,  # 20% warmup
        anneal_strategy='cos',
        div_factor=train_config.div_factor,
        final_div_factor=train_config.final_div_factor,
        three_phase=True
    )
    # Setup validation loss smoother
    val_smoother = MetricSmoother(window_size=5)

    # Training loop
    best_val_loss = float('inf')
    patience = 0

    train_acc = 0.0
    val_acc = 0.0
    avg_grad_norm = 0.0

    for epoch in range(train_config.epochs):
        print(f'\nEpoch {epoch + 1}/{train_config.epochs}')

        # Training phase
        train_loss, train_acc, mixup_alpha, avg_grad_norm = __train_epoch(
            model=model,
            loader=train_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            grad_clip_value=train_config.grad_clip_value,
            label_smoothing=train_config.label_smoothing,
            prev_train_acc=train_acc,
            prev_val_acc=val_acc,
            epoch=epoch,
            total_epochs=train_config.epochs,
            avg_grad_norm=avg_grad_norm
        )

        # Validation phase
        val_loss, val_acc = __validate(model, val_loader)
        smoothed_val_loss = val_smoother.update(val_loss)

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train/loss": train_loss,
            "train/accuracy": train_acc,
            "val/loss": val_loss,
            "val/accuracy": val_acc,
            "val/smoothed_loss": smoothed_val_loss,
            "metrics/acc_gap": train_acc - val_acc,
            "learning_rate": optimizer.param_groups[0]["lr"],
            "mixup_alpha": mixup_alpha,
            "avg_grad_norm": avg_grad_norm
        })

        # Logging
        print(
            f'Epoch {epoch + 1}: '
            f'Train Loss: {train_loss:.4f}, '
            f'Train Acc: {train_acc:.4f}, '
            f'Val Loss: {val_loss:.4f}, '
            f'Val Acc: {val_acc:.4f}, '
            f"Smoothed Val Loss: {smoothed_val_loss:.4f}, "
            f'Gap: {(train_acc - val_acc):.4f}, '
            f'LR: {optimizer.param_groups[0]["lr"]:.6f}'
        )

        # Model checkpointing
        if smoothed_val_loss < best_val_loss:
            best_val_loss = smoothed_val_loss
            patience = 0
            torch.save(model.state_dict(), f'model_{node_id}.pth')
            print(f'Saved new best model with val_loss: {val_loss:.4f}')

            wandb.save(f'model_{node_id}.pth')
            wandb.run.summary["best_val_loss"] = best_val_loss
            wandb.run.summary["best_val_acc"] = val_acc
        else:
            patience += 1
            if patience >= train_config.early_stopping_patience:
                print('Early stopping triggered')
                wandb.run.summary["early_stopping_epoch"] = epoch + 1
                break

    print('Training completed')
    wandb.finish()



In [14]:
config = TrainConfig(
        epochs=150,
        batch_size=16,
        max_lr=1.2e-3,
        div_factor=15.0,
        final_div_factor=100.0,
        pct_start=0.4,
        weight_decay=0.1,    # Increased for more regularization
        grad_clip_value=0.4,
        label_smoothing=0.1,
        early_stopping_patience=30,
        optimizer="adamw",
    )

In [15]:
hierarchy = Hierarchy()

In [16]:
torch.serialization.add_safe_globals([Image])

In [20]:
root_id = "n03862676"

In [21]:
train_singular_model(hierarchy, root_id, config, "cuda")

wandb: Currently logged in as: hedrekao (hedrekao-via-university-college). Use `wandb login --relogin` to force relogin


n03259280: 337 samples (51.06%)
n04111531: 323 samples (48.94%)
Skipping balanced class n03259280, already has 337 samples
Balanced class n04111531 to 337 samples
n03259280: 72 samples (51.06%)
n04111531: 69 samples (48.94%)

Epoch 1/150
Mixup alpha: 0.44000000000000006


Training:   0%|          | 0/42 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1: Train Loss: 0.6927, Train Acc: 0.5327, Val Loss: 0.6891, Val Acc: 0.5469, Smoothed Val Loss: 0.6891, Gap: -0.0141, LR: 0.000081
Saved new best model with val_loss: 0.6891

Epoch 2/150
Mixup alpha: 0.66


...

Epoch 149: Train Loss: 0.5645, Train Acc: 0.8452, Val Loss: 0.4790, Val Acc: 0.7578, Smoothed Val Loss: 0.4785, Gap: 0.0874, LR: 0.000001

Epoch 150/150
Mixup alpha: 0.6


Training:   0%|          | 0/42 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 150: Train Loss: 0.5798, Train Acc: 0.8036, Val Loss: 0.4783, Val Acc: 0.7734, Smoothed Val Loss: 0.4781, Gap: 0.0301, LR: 0.000001
Training completed


avg_grad_norm,▁▂▁▂▂▂▂▃▄▃▃▆▃▆▅▅▆▆██▄▆▅▆▆▆▅▅▅▅▆▅▅▆▅▅▅▅▄▄
epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
learning_rate,▂▂▂▂▂▃▄▄▄▅▆▇▇▇█████▇▆▅▅▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁
metrics/acc_gap,▃▃▄▆▄▆▄▃▅█▄▂▃▃▇▂▅▁█▆█▃▆▄▆▁▇▆▅▃▆▅▄▅▃▅▄▃▅▄
mixup_alpha,█▅▅█▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,▁▂▂▂▁▃▂▃▄▄▃▃▄▃▄▄▅▅▃▅▅▅▅▅▆▆▆▅▇▆▆▇▇▇▆█▇███
train/loss,█████▇▆▇▇▆▆▇▆▆▇▆▆▆▆▅▄▄▄▅▄▃▂▄▃▂▃▂▂▃▂▂▁▂▂▂
val/accuracy,▁▁▂▃▁▂▃▃▃▄▅▄▅▃▄▅▃▅▃▅▆▆▆▆▆▇▆▇█▇█████▇████
val/loss,▆▆▆▅▆▆▅▄█▅▄▅▄▄▃▇▃▃▄▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val/smoothed_loss,██▇▇▇▇▆▆▆█▆▅▅▅▅▇▇▄▄▄▄▄▄▄▄▃▃▃▃▃▂▁▁▁▁▁▁▁▁▁
avg_grad_norm,2.50995


In [22]:
state_dict = torch.load(f'model_{root_id}.pth')

children = hierarchy.get_children(root_id)
model = HierarchyNodeModel(len(children))

model.load_state_dict(state_dict)
model.to("cuda")

categories_dict = {child: hierarchy.get_leaf_nodes(
        child) for child in children}

test_loader = create_images_dataloader(
        categories_dict, split='test', batch_size=config.batch_size)

test_loss, test_acc = __validate(model, test_loader)

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

n03259280: 73 samples (51.05%)
n04111531: 70 samples (48.95%)


<ipython-input-22-8deb53235062>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f'model_{root_id}.pth')


Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Test Loss: 0.3976, Test Acc: 0.8516
